# Week 7: Natural language processing

The exercises today are about extracting high-level knowledge from text. We're still a long way from computers being able to give us insight as deep as that which we can aquire from manually reading text, but some the tools that you will use today get us a long way in understanding useful things about unreadibly large amount of text in comparatively little time. In the exercises today you will:

* Create wordclouds
* Extract sentiment from text
* Construct a Bag of Words (BoW) matrix to represent how words are used about each faction in the Marvel dataset
* Perform a TD-IDF transform to understand which words are important to different characters

[**Feedback**](http://ulfaslak.com/vent)

## Exercises

### Part 1 (Extra): Wordcloud

Although they probably offer more buzz than usefulness, wordclouds are a fun way to get quick insight into which words are used in a corpus of text. In this section you will generate one.

>**Ex. 7.1.1**: To make a word cloud you need some more or less clean text. For each character extract as cleanly as you can, the text written on their wikipage. Since the wikidata is kind of messy, there are some things you should be aware of when extracting the text.
* Exclude character names
* Exclude links
* Exclude references
* Exclude numbers
* Set everything to lower case
* Do not include stopwords (use `nltk.corpus.stopwords.words("english")` to get a list of stopwords; install `nltk` to do this)

>Cleaning doesn't have to be perfect and can be done in a many different ways, these are just some things to look out for.

>Once you have extracted the text, create one long text string for all text written about heroes, another long text string for villains, and finally one for ambiguous characters. Using the code snippet below which shows how to plot a word cloud, plot the word clouds for each faction.

>        text = "some cool text"
>        wc = wordcloud.WordCloud(max_font_size=40).generate(text)
>        
>        plt.figure()
>        plt.imshow(wc, interpolation="bilinear")
>        plt.axis("off")
>        plt.show()

> You have to import `matplotlib.pylab` and `wordcloud` to do this. You can install `wordcloud` with anaconda by typing into your console

>        conda install -c conda-forge wordcloud

>or

>        pip install wordcloud

>if you don't have Anaconda installed.

Helper function to get vector of words for a character. Make sure the path to your data is correct.

In [61]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# Get list of stopwords and add a few
stop_words = set(stopwords.words('english'))
stop_words.update(['ref', 'ii', 'comics'])

def get_clean_words(title, folder):
    def _isnum(w):
        try:
            int(w)
            return True
        except ValueError:
            return False
        
    # Load her markup
    with open(f"../data/{folder}/{title}.txt") as fp:
        markup = fp.read()
        
    # Remove table and external links
    markup = re.sub(r'\{\{[\s\S]*?\}\}', '', markup)

    # Remove category links
    text = re.sub(r'\[\[Category.+\]\]', '', markup)
    
    # Remove own title
    text = text.replace(title, "")

    # Extarct words
    words = word_tokenize(text)
    
    # Convert to lower case
    words = [w.lower() for w in words]
    
    # Clear punctuation
    table = str.maketrans('', '', string.punctuation)
    words = [w.translate(table) for w in words]
    
    # Remove non-alphabetic words
    words = [w for w in words if w.isalpha()]
    
    # Remove stop words
    words = [w for w in words if not w in stop_words]
    
    # Remove single letter words 
    words = [w for w in words if len(w) > 1]

    return words

# Example
get_clean_words("Iron Man", "heroes")

['fictional',
 'superhero',
 'appearing',
 'american',
 'comic',
 'book',
 'published',
 'marvel',
 'character',
 'cocreated',
 'writer',
 'editor',
 'stan',
 'lee',
 'developed',
 'scripter',
 'larry',
 'lieber',
 'designed',
 'artists',
 'heck',
 'jack',
 'kirby',
 'character',
 'made',
 'first',
 'appearance',
 'tales',
 'suspense',
 'cover',
 'date',
 'march',
 'received',
 'title',
 'comic',
 'book',
 'may',
 'wealthy',
 'american',
 'business',
 'magnate',
 'playboy',
 'lifestyle',
 'playboy',
 'ingenious',
 'scientist',
 'anthony',
 'edward',
 'tony',
 'stark',
 'suffers',
 'severe',
 'chest',
 'injury',
 'kidnapping',
 'captors',
 'attempt',
 'force',
 'build',
 'weapon',
 'mass',
 'destruction',
 'instead',
 'creates',
 'armormechanized',
 'suit',
 'armor',
 'save',
 'life',
 'escape',
 'captivity',
 'later',
 'stark',
 'develops',
 'suit',
 'adding',
 'weapons',
 'technological',
 'devices',
 'designed',
 'company',
 'stark',
 'industries',
 'uses',
 'suit',
 'successive',
 '

### Part 2: Sentiment analysis

Often you don't have time to read some text, but you need to know something about whether it positively or negatively toned. Enter *Sentiment Analysis*. The point of this exercise is to extract the sentiment of text on your heroes, villains and ambiguous characters and figure out whether Wikipedia is biased towards writing in a certain tone towards a certain kind of characters.

>**Ex. 7.2.1**: Compute the sentiment score of each character's page and produce three histograms of sentiment scores, one for each faction. You can use the text strings you generated in the previous exercise. We will cheat a bit and use a library that does the scoring for us. Install `afinn` using `conda` or `pip` and extract the sentiment with that module. There's an example of how to use it on the library's [PyPi repository](https://pypi.python.org/pypi/afinn).

### Part 3: Bag of Words matrix

Intuitively, "Bag of Words" means breaking up a document into words and throwing them into a bag. And that's very close to the truth! In week 5 you constructed a "team-affiliations" matrix which had a row for each character and a column for each team. If the character was on a given team there would be a one for that character row at that team column, if not there would be a zero. The BoW is the same, only now, rather than teams, your columns are individual words that a character's wikipage might contain, and the numbers represent how many times those words appear.
<img src="http://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_bow.png" width="400"/>
BoW's are pretty large and sparse (mostly contain zero's) matrices, but they are extremely useful because they allow us to use linear algebra to do things like PCA, classification, etc..

>**Ex. 7.3.1**: Create a bag of words matrix that contains all your characters. Maintain also a target array, so you know whether a row corresponds to a hero, a villain or an ambiguous character. Also, **do not include words that only appear for one character**.
1. How many different words are in your vocabulary/columns are in your matrix?
2. Print the 10 most used words, and the 10 least used words, along with their usage count.
3. Plot the distribution of how many times words are used.

>*Hint: Since you already extracted lists of words for each faction in Ex. 7.1.1, you can use these to figure out what the total vocabulary of words used in your dataset is. You can "clean up" this vocabulary by a number of tricks. For example, there are tools for *stemming* words to remove grammar so that e.g. 'cat' and 'cats' both become 'cat', but that's all up to you whether you wanna go that deep.*

> **Ex. 7.3.EXTRA**: Plot the first two components of the PCA transform of your BoW matrix. See anything interesting?

> **Ex. 7.3.EXTRA**: Create a classifier that predicts if a character is a hero or a villain from the words used on their page. Report its 10-fold cross validation accuracy.

### Part 4: TF-IDF

You didn't just make that BoW matrix to count simple things. We are interested in knowing how (or if) words are used differently across characters, and the best way to do that is to used something called a Term Frequency - Inverse Document Frequency (TF-IDF) transformation. You can read about it [on Wikipedia](https://en.wikipedia.org/wiki/Tf%E2%80%93idf), but the gist is that it reflects how important each word is to each document (a document in this case being a faction).

It works in two steps:
* (1 - TF) you normalize over the frequency of each word in each document, so that rows sum to 1. Every row is now a probability distribution (a pmf to be exact) that gives the "term frequency" in each document.
* (2 - IDF) you weigh the TF by the inverse document frequency, which measures how unique a word to specific documents. For example, the word "the" will be frequently used in every document (high TF) but we know it's not very special because it's used in all documents so the inverse document frequency is low, yielding a vanishing TF-IDF score for "the" in all documents.

>**Ex. 7.4.1**: Use any tool you like (you can do it manually, it's straight forward if you understand the method), to perform a TF-IDF transform on your BoW matrix from Ex. 7.3.1. The result should be a matrix of the same shape as the BoW, but with different values inside.
1. Explain what these values mean.
2. For the top three most written about characters in each class (so 9 in total), print out each of their 10 highest scoring words. Comment on any differences you observe in the type of words being used in different classes.

>**Ex. 7.4.2**: Import `from sklearn.manifold import TSNE` and transform the BoW matrix into a lower-dimensional space. Plot the points in this space.
>
>*Hint: You can simply do `X_tsne = TSNE(n_components=2).fit_transform(X_BoW)` and then plot the 0th and 1st axis of `X_tsne` against each other. If there are clusters of words use in the data, this plot will show them. You can make this even cooler by coloring the scattered points by their class.*